In [ ]:
%matplotlib inline
from agents.trafficagent import TrafficAgent, FixedCycleTrafficAgent, QLearningTrafficAgent, DeepQLearningTrafficAgent
from utils.plotter import Metric, MultiPlotter

seconds = 100

agents: dict[str, TrafficAgent] = {
  'fixedcycle': FixedCycleTrafficAgent(
    'fixedcycle',
    '#aa0000',
    'nets/single-intersection/single-intersection.net.xml',
    'nets/single-intersection/single-intersection.rou.xml',
    seconds,
    5,
    3,
    5,
    30
  ),
  'qlearning': QLearningTrafficAgent(
    'qlearning',
    '#00aa00',
    'nets/single-intersection/single-intersection.net.xml',
    'nets/single-intersection/single-intersection.rou.xml',
    seconds,
    5,
    3,
    5,
    30
  ),
  'deepqlearning': DeepQLearningTrafficAgent(
    'deepqlearning',
    '#0000aa',
    'nets/single-intersection/single-intersection.net.xml',
    'nets/single-intersection/single-intersection.rou.xml',
    seconds,
    5,
    3,
    5,
    30
  )
}
metrics: list[Metric] = [
  'system_total_stopped',
  'system_total_waiting_time',
  'system_mean_waiting_time',
  'system_mean_speed',
  't_stopped',
  't_accumulated_waiting_time',
  't_average_speed',
  'agents_total_stopped',
  'agents_total_accumulated_waiting_time'
]

multi_plotter = MultiPlotter(
  list(map(lambda agent: {'name': agent.name, 'color': agent.color}, list(agents.values()))),
  metrics,
  seconds,
  1
)

for agent in agents:
  agents[agent].learn(multi_plotter.append)

multi_plotter.save()